# 🏈 NFL Full Pick Generator — v3.2

**New in v3.2**
- Week **Override** (or leave on **Auto**)
- **Quick‑Start** block at the top
- Keeps: auto injuries (Out/Doubtful), weather → totals only, sortable & color‑coded tables in Colab, 3‑tab Excel export with download link, summary of top edges

Use on iPad in **Google Colab**: open → **Runtime → Run all**.


## 🚀 Quick‑Start
1) To run **Auto week**: just do **Runtime → Run all**.

2) To **override week**:
- Change the dropdown below from **Auto** to the week you want, then **Runtime → Run all**.


In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML
    weeks = ['Auto'] + [str(i) for i in range(1, 23)]
    week_dropdown = widgets.Dropdown(options=weeks, value='Auto', description='Week:')
    display(week_dropdown)
except Exception as e:
    week_dropdown = None
    print('Widgets unavailable; defaulting to Auto week.')


In [ ]:
!pip -q install pandas openpyxl lxml beautifulsoup4 requests

In [ ]:
import pandas as pd, numpy as np, io, re, datetime, requests, math
from bs4 import BeautifulSoup
from IPython.display import display, HTML
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

try:
    from google.colab import data_table
    data_table.enable_dataframe_formatter()
    COLAB = True
except Exception:
    COLAB = False


## ⚙️ Settings (editable)

In [ ]:
# Home field advantage (points) — spreads only
HOME_FIELD_ADV = 1.5

# Weather thresholds
WIND_THRESHOLD = 15.0     # mph
PRECIP_THRESHOLD = 50     # % probability
TOTALS_WEATHER_ADJ = 2.5  # points subtracted when threshold hit

# Position-based injury adjustments (editable)
position_adjustments = {
    'QB': -2.0,
    'LT': -0.7, 'RT': -0.7, 'OT': -0.7,
    'LG': -0.5, 'RG': -0.5, 'C': -0.5, 'G': -0.5,
    'WR': -0.5, 'TE': -0.5,
    'RB': -0.3,
    'CB': -0.5, 'S': -0.4,
    'LB': -0.3,
    'EDGE': -0.4, 'DE': -0.4, 'DT': -0.3, 'DL': -0.3,
}

# Weights for Power Rating (z-scored metrics)
WEIGHTS = {
    'Off_EPA_per_play': 0.25,
    'Def_EPA_per_play': 0.25,
    'Off_Success_Rate': 0.15,
    'Def_Success_Rate': 0.15,
    'Turnover_Margin': 0.10,
    'Red_Zone_Off_Eff': 0.05,
    'Red_Zone_Def_Eff': 0.05,
    'Adj_Sack_Rate_Off': 0.025,
    'Adj_Sack_Rate_Def': 0.025,
}


## 🧰 Helpers

In [ ]:
def zscore(col):
    c = col.astype(float)
    std = c.std(ddof=0)
    return (c - c.mean()) / (std if std != 0 else 1.0)

def team_norm(name):
    mapping = {
        'LA Rams':'Los Angeles Rams','L.A. Rams':'Los Angeles Rams','LAR':'Los Angeles Rams',
        'LA Chargers':'Los Angeles Chargers','L.A. Chargers':'Los Angeles Chargers','LAC':'Los Angeles Chargers',
        'Washington Redskins':'Washington Commanders','Washington Football Team':'Washington Commanders','WSH':'Washington Commanders',
        'Oakland Raiders':'Las Vegas Raiders','LV':'Las Vegas Raiders','LVR':'Las Vegas Raiders',
        'JAX':'Jacksonville Jaguars','TB':'Tampa Bay Buccaneers','NO':'New Orleans Saints',
        'NE':'New England Patriots','SF':'San Francisco 49ers','DAL':'Dallas Cowboys','NYJ':'New York Jets','NYG':'New York Giants',
        'KC':'Kansas City Chiefs','BUF':'Buffalo Bills','PHI':'Philadelphia Eagles','MIA':'Miami Dolphins','GB':'Green Bay Packers',
        'TEN':'Tennessee Titans','CLE':'Cleveland Browns','CIN':'Cincinnati Bengals','PIT':'Pittsburgh Steelers','BAL':'Baltimore Ravens','HOU':'Houston Texans',
        'IND':'Indianapolis Colts','CHI':'Chicago Bears','DET':'Detroit Lions','MIN':'Minnesota Vikings','ATL':'Atlanta Falcons',
        'CAR':'Carolina Panthers','SEA':'Seattle Seahawks','ARI':'Arizona Cardinals','DEN':'Denver Broncos',
    }
    return mapping.get(str(name).strip(), str(name).strip())

def safe_get(url, headers=None, params=None):
    headers = headers or {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36'}
    r = requests.get(url, headers=headers, params=params, timeout=45)
    r.raise_for_status()
    return r

def autodetect_season_week():
    url = 'https://raw.githubusercontent.com/nflverse/nflfastR-data/master/schedules.csv'
    sched = pd.read_csv(io.BytesIO(safe_get(url).content))
    sched = sched[sched['game_type'].isin(['REG','POST'])].copy()
    sched['start_time'] = pd.to_datetime(sched['start_time'], utc=True)
    now = pd.Timestamp.utcnow()
    window = (sched['start_time'] >= now - pd.Timedelta(days=4)) & (sched['start_time'] <= now + pd.Timedelta(days=6))
    near = sched[window]
    if not near.empty:
        grp = near.groupby(['season','week','game_type']).size().reset_index(name='n')
        row = grp.sort_values('n', ascending=False).iloc[0]
        return int(row['season']), int(row['week']), row['game_type']
    upcoming = sched[sched['start_time'] >= now]
    if not upcoming.empty:
        row = upcoming.sort_values('start_time').iloc[0]
        return int(row['season']), int(row['week']), row['game_type']
    last = sched.sort_values('start_time').iloc[-1]
    return int(last['season']), int(last['week']), last['game_type']

def apply_week_override(auto_season_week):
    season, week, gtype = auto_season_week
    try:
        sel = (week_dropdown.value if week_dropdown else 'Auto')
    except Exception:
        sel = 'Auto'
    if sel and str(sel).lower()!='auto':
        try:
            week = int(sel)
        except Exception:
            pass
    return season, week, gtype

def fetch_schedule(season, week, game_type='REG'):
    url = 'https://raw.githubusercontent.com/nflverse/nflfastR-data/master/schedules.csv'
    sched = pd.read_csv(io.BytesIO(safe_get(url).content))
    sched = sched[(sched['season']==season) & (sched['week']==week) & (sched['game_type'].isin([game_type,'REG']))].copy()
    sched['start_time'] = pd.to_datetime(sched['start_time'], utc=True)
    for c in ['home_team','away_team']:
        sched[c] = sched[c].map(team_norm)
    return sched[['game_id','season','week','game_type','start_time','home_team','away_team','stadium','location']]

# Stadium mapping (roof + coords)
STADIUMS = {
  'GEHA Field at Arrowhead Stadium': {'lat':39.0490,'lon':-94.4839,'roof':'outdoor','team':'Kansas City Chiefs'},
  'Highmark Stadium': {'lat':42.7738,'lon':-78.7869,'roof':'outdoor','team':'Buffalo Bills'},
  'Gillette Stadium': {'lat':42.0909,'lon':-71.2643,'roof':'outdoor','team':'New England Patriots'},
  'MetLife Stadium': {'lat':40.8136,'lon':-74.0745,'roof':'outdoor','team':'New York Jets'},
  'M&T Bank Stadium': {'lat':39.2779,'lon':-76.6229,'roof':'outdoor','team':'Baltimore Ravens'},
  'Paycor Stadium': {'lat':39.0954,'lon':-84.5160,'roof':'outdoor','team':'Cincinnati Bengals'},
  'Cleveland Browns Stadium': {'lat':41.5061,'lon':-81.6995,'roof':'outdoor','team':'Cleveland Browns'},
  'Acrisure Stadium': {'lat':40.4468,'lon':-80.0158,'roof':'outdoor','team':'Pittsburgh Steelers'},
  'NRG Stadium': {'lat':29.6847,'lon':-95.4107,'roof':'retractable','team':'Houston Texans'},
  'Lucas Oil Stadium': {'lat':39.7601,'lon':-86.1639,'roof':'retractable','team':'Indianapolis Colts'},
  'EverBank Stadium': {'lat':30.3239,'lon':-81.6372,'roof':'outdoor','team':'Jacksonville Jaguars'},
  'Nissan Stadium': {'lat':36.1665,'lon':-86.7713,'roof':'outdoor','team':'Tennessee Titans'},
  'Empower Field at Mile High': {'lat':39.7439,'lon':-105.0201,'roof':'outdoor','team':'Denver Broncos'},
  'Caesars Superdome': {'lat':29.9511,'lon':-90.0812,'roof':'dome','team':'New Orleans Saints'},
  'Bank of America Stadium': {'lat':35.2258,'lon':-80.8528,'roof':'outdoor','team':'Carolina Panthers'},
  'Raymond James Stadium': {'lat':27.9759,'lon':-82.5033,'roof':'outdoor','team':'Tampa Bay Buccaneers'},
  'Mercedes-Benz Stadium': {'lat':33.7554,'lon':-84.4008,'roof':'retractable','team':'Atlanta Falcons'},
  'Hard Rock Stadium': {'lat':25.9580,'lon':-80.2389,'roof':'outdoor','team':'Miami Dolphins'},
  'Soldier Field': {'lat':41.8623,'lon':-87.6167,'roof':'outdoor','team':'Chicago Bears'},
  'U.S. Bank Stadium': {'lat':44.9738,'lon':-93.2578,'roof':'dome','team':'Minnesota Vikings'},
  'Ford Field': {'lat':42.3400,'lon':-83.0456,'roof':'dome','team':'Detroit Lions'},
  'Lambeau Field': {'lat':44.5013,'lon':-88.0622,'roof':'outdoor','team':'Green Bay Packers'},
  'AT&T Stadium': {'lat':32.7473,'lon':-97.0945,'roof':'retractable','team':'Dallas Cowboys'},
  'Lincoln Financial Field': {'lat':39.9008,'lon':-75.1675,'roof':'outdoor','team':'Philadelphia Eagles'},
  'FedExField': {'lat':38.9077,'lon':-76.8645,'roof':'outdoor','team':'Washington Commanders'},
  'SoFi Stadium': {'lat':33.9535,'lon':-118.3392,'roof':'dome','team':'Los Angeles Rams'},
  'Levi\'s Stadium': {'lat':37.4030,'lon':-121.9696,'roof':'outdoor','team':'San Francisco 49ers'},
  'Lumen Field': {'lat':47.5952,'lon':-122.3316,'roof':'outdoor','team':'Seattle Seahawks'},
  'State Farm Stadium': {'lat':33.5276,'lon':-112.2626,'roof':'retractable','team':'Arizona Cardinals'},
  'Allegiant Stadium': {'lat':36.0908,'lon':-115.1832,'roof':'dome','team':'Las Vegas Raiders'},
}

TEAM_HOME = {
  'Arizona Cardinals': 'State Farm Stadium',
  'Atlanta Falcons': 'Mercedes-Benz Stadium',
  'Baltimore Ravens': 'M&T Bank Stadium',
  'Buffalo Bills': 'Highmark Stadium',
  'Carolina Panthers': 'Bank of America Stadium',
  'Chicago Bears': 'Soldier Field',
  'Cincinnati Bengals': 'Paycor Stadium',
  'Cleveland Browns': 'Cleveland Browns Stadium',
  'Dallas Cowboys': 'AT&T Stadium',
  'Denver Broncos': 'Empower Field at Mile High',
  'Detroit Lions': 'Ford Field',
  'Green Bay Packers': 'Lambeau Field',
  'Houston Texans': 'NRG Stadium',
  'Indianapolis Colts': 'Lucas Oil Stadium',
  'Jacksonville Jaguars': 'EverBank Stadium',
  'Kansas City Chiefs': 'GEHA Field at Arrowhead Stadium',
  'Las Vegas Raiders': 'Allegiant Stadium',
  'Los Angeles Chargers': 'SoFi Stadium',
  'Los Angeles Rams': 'SoFi Stadium',
  'Miami Dolphins': 'Hard Rock Stadium',
  'Minnesota Vikings': 'U.S. Bank Stadium',
  'New England Patriots': 'Gillette Stadium',
  'New Orleans Saints': 'Caesars Superdome',
  'New York Giants': 'MetLife Stadium',
  'New York Jets': 'MetLife Stadium',
  'Philadelphia Eagles': 'Lincoln Financial Field',
  'Pittsburgh Steelers': 'Acrisure Stadium',
  'San Francisco 49ers': "Levi's Stadium",
  'Seattle Seahawks': 'Lumen Field',
  'Tampa Bay Buccaneers': 'Raymond James Stadium',
  'Tennessee Titans': 'Nissan Stadium',
  'Washington Commanders': 'FedExField',
}

def open_meteo_weather(lat, lon, kickoff):
    base='https://api.open-meteo.com/v1/forecast'
    params={'latitude':lat,'longitude':lon,'hourly':['windspeed_10m','precipitation_probability'],'timezone':'UTC'}
    r = requests.get(base, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()
    times = pd.to_datetime(data['hourly']['time'], utc=True)
    ws = pd.Series(data['hourly']['windspeed_10m'], index=times)
    pp = pd.Series(data['hourly']['precipitation_probability'], index=times)
    win = (times >= kickoff - pd.Timedelta(hours=3)) & (times <= kickoff + pd.Timedelta(hours=3))
    if win.sum()==0:
        return None
    wind_mph = ws[win].max() * 0.621371
    precip_prob = float(pp[win].max())
    return {'max_wind_mph': float(wind_mph), 'max_precip_prob': precip_prob}

# NFL.com injury scraping (Out/Doubtful only)
def normalize_pos(p):
    p = str(p).upper().strip()
    if p in ['LT','RT','OT','T']: return 'OT' if p=='T' else p
    if p in ['LG','RG','C','G','OL']: return p if p!='OL' else 'G'
    if p in ['DE','DT','DL','NT','EDGE']: return 'DL' if p!='EDGE' else 'EDGE'
    if p in ['SS','FS','S']: return 'S'
    return p

def pos_value(p):
    return position_adjustments.get(normalize_pos(p), 0.0)

def fetch_nfl_injuries_week(season, week):
    urls = [
        f'https://www.nfl.com/injuries/league/{season}/REG{week}',
        f'https://www.nfl.com/injuries/league/{season}/week/{week}',
        'https://www.nfl.com/injuries/league'
    ]
    rows=[]
    for url in urls:
        try:
            html = safe_get(url).text
        except Exception:
            continue
        soup = BeautifulSoup(html, 'lxml')
        for tbl in soup.find_all('table'):
            headers = [th.get_text(strip=True) for th in tbl.find_all('th')]
            if not headers or not any('Player' in h for h in headers):
                continue
            cur_team=None
            for tr in tbl.find_all('tr'):
                ths = [th.get_text(strip=True) for th in tr.find_all('th')]
                tds = [td.get_text(strip=True) for td in tr.find_all('td')]
                if len(tds)==0 and len(ths)==1:
                    cur_team = team_norm(ths[0]); continue
                if len(tds)>=3:
                    row = {headers[i]:tds[i] for i in range(min(len(headers),len(tds)))}
                    team = row.get('Team') or cur_team
                    player = row.get('Player') or tds[0]
                    pos = row.get('Pos') or row.get('Position') or (tds[1] if len(tds)>1 else '')
                    status = row.get('Game Status') or row.get('Status') or tds[-1]
                    if team:
                        rows.append([team_norm(team), player, normalize_pos(pos), status])
        if rows: break
    if not rows:
        return pd.DataFrame(columns=['Team','Player','Pos','Status','Adj'])
    df = pd.DataFrame(rows, columns=['Team','Player','Pos','Status'])
    df['StatusLow']=df['Status'].str.lower()
    df = df[df['StatusLow'].str.contains('out|doubtful', na=False)].copy()
    df['Adj']=df['Pos'].map(pos_value)
    return df[['Team','Player','Pos','Status','Adj']]


## 📅 1) Auto‑detect season/week (with override) & load schedule

In [ ]:
SEASON, WEEK, GAME_TYPE = autodetect_season_week()
SEASON, WEEK, GAME_TYPE = apply_week_override((SEASON, WEEK, GAME_TYPE))
print(f'Using → Season {SEASON}, Week {WEEK}, Type {GAME_TYPE}')
schedule = fetch_schedule(SEASON, WEEK, GAME_TYPE)
display(schedule.head())


## 📊 2) Fetch core team stats

In [ ]:
def fetch_rbsdm_epa_success(season):
    try:
        url = f'https://rbsdm.com/stats/teams/season_team_{season}.csv'
        df = pd.read_csv(io.BytesIO(safe_get(url).content))
        mapping = {}
        for c in df.columns:
            lc = c.lower()
            if 'team' in lc and 'abbr' not in lc: mapping['Team']=c
            if 'epa' in lc and 'off' in lc and 'play' in lc: mapping['Off_EPA_per_play']=c
            if 'epa' in lc and 'def' in lc and 'play' in lc: mapping['Def_EPA_per_play']=c
            if ('success' in lc or 'sr' in lc) and 'off' in lc: mapping['Off_Success_Rate']=c
            if ('success' in lc or 'sr' in lc) and 'def' in lc: mapping['Def_Success_Rate']=c
        out = df.rename(columns={v:k for k,v in mapping.items()})[['Team','Off_EPA_per_play','Def_EPA_per_play','Off_Success_Rate','Def_Success_Rate']]
        out['Team']=out['Team'].map(team_norm)
        return out
    except Exception:
        teams = sorted(TEAM_HOME.keys())
        return pd.DataFrame({'Team':teams,'Off_EPA_per_play':0.0,'Def_EPA_per_play':0.0,'Off_Success_Rate':0.0,'Def_Success_Rate':0.0})

def fetch_pfr_turnover_margin(season):
    try:
        url = f'https://www.pro-football-reference.com/years/{season}/'
        dfs = pd.read_html(safe_get(url).text)
        cand=None
        for d in dfs:
            cols=[str(c).lower() for c in d.columns]
            if any('team' in c or 'tm' in c for c in cols) and any('tom' in c or 'margin' in c or 'turnover' in c for c in cols):
                cand=d;break
        if cand is None: raise ValueError
        team_col = next((c for c in cand.columns if 'team' in str(c).lower() or 'tm' in str(c).lower()), cand.columns[0])
        tom = next((c for c in cand.columns if 'margin' in str(c).lower() or 'tom' in str(c).lower()), None)
        if tom is not None:
            out=cand[[team_col,tom]].copy(); out.columns=['Team','Turnover_Margin']
        else:
            give = next((c for c in cand.columns if 'give' in str(c).lower()), None)
            take = next((c for c in cand.columns if 'take' in str(c).lower()), None)
            out=cand[[team_col,give,take]].copy(); out.columns=['Team','Give','Take']
            out['Turnover_Margin']=out['Take'].astype(float)-out['Give'].astype(float)
            out=out[['Team','Turnover_Margin']]
        out['Team']=out['Team'].str.replace('*','',regex=False).map(team_norm)
        return out
    except Exception:
        teams = sorted(TEAM_HOME.keys())
        return pd.DataFrame({'Team':teams,'Turnover_Margin':0.0})

def fetch_teamrankings_redzone(off=True):
    try:
        url = 'https://www.teamrankings.com/nfl/stat/red-zone-scoring-pct' if off else 'https://www.teamrankings.com/nfl/stat/opponent-red-zone-scoring-pct'
        dfs = pd.read_html(safe_get(url).text)
        df = dfs[0].copy()
        val_col = next((c for c in df.columns if c!='Team'), df.columns[1])
        out = df[['Team',val_col]].copy(); out.columns=['Team','val']
        out['Team']=out['Team'].map(team_norm)
        return out
    except Exception:
        teams = sorted(TEAM_HOME.keys())
        return pd.DataFrame({'Team':teams,'val':0.0})

def fetch_adjusted_sack_rate():
    teams = sorted(TEAM_HOME.keys())
    return pd.DataFrame({'Team':teams,'Adj_Sack_Rate_Off':0.0,'Adj_Sack_Rate_Def':0.0})

epa_sr = fetch_rbsdm_epa_success(SEASON)
tom    = fetch_pfr_turnover_margin(SEASON)
rzo    = fetch_teamrankings_redzone(True).rename(columns={'val':'Red_Zone_Off_Eff'})
rzd    = fetch_teamrankings_redzone(False).rename(columns={'val':'Red_Zone_Def_Eff'})
asr    = fetch_adjusted_sack_rate()
stats = epa_sr.merge(tom, on='Team', how='outer').merge(rzo, on='Team', how='outer').merge(rzd, on='Team', how='outer').merge(asr, on='Team', how='outer').fillna(0.0)
display(stats.head())


## 🧮 3) Base Power Ratings

In [ ]:
pr = stats.copy()
for col in WEIGHTS.keys(): pr[col+'_z'] = zscore(pr[col])
pr['Base_Power_Rating'] = sum(pr[c+'_z']*w for c,w in WEIGHTS.items())
ratings = pr[['Team','Base_Power_Rating','Off_EPA_per_play','Def_EPA_per_play']].copy()
display(ratings.sort_values('Base_Power_Rating', ascending=False).head())


## 🚑 4) Auto Injuries → Team adjustments + details (Out/Doubtful only)

In [ ]:
inj_details = fetch_nfl_injuries_week(SEASON, WEEK)
team_inj = inj_details.groupby('Team')['Adj'].sum().reset_index().rename(columns={'Adj':'Injury_Adjustment'}) if not inj_details.empty else pd.DataFrame(columns=['Team','Injury_Adjustment'])
ratings = ratings.merge(team_inj, on='Team', how='left')
ratings['Injury_Adjustment'] = ratings['Injury_Adjustment'].fillna(0.0)
ratings['Final_Power_Rating'] = ratings['Base_Power_Rating'] + ratings['Injury_Adjustment']
display(ratings.sort_values('Final_Power_Rating', ascending=False).head())
display(inj_details.head())


## 📈 5) Projections (weather → totals only)

In [ ]:
idx = ratings.set_index('Team')
rows = []
for _, g in fetch_schedule(SEASON, WEEK, GAME_TYPE).iterrows():
    home, away = g['home_team'], g['away_team']
    if home not in idx.index or away not in idx.index: continue
    spread_base  = (idx.loc[home,'Base_Power_Rating']  - idx.loc[away,'Base_Power_Rating'])  + HOME_FIELD_ADV
    spread_final = (idx.loc[home,'Final_Power_Rating'] - idx.loc[away,'Final_Power_Rating']) + HOME_FIELD_ADV
    injury_shift = float(spread_final - spread_base)
    # Totals heuristic
    home_off, away_off = float(idx.loc[home,'Off_EPA_per_play']), float(idx.loc[away,'Off_EPA_per_play'])
    home_def, away_def = float(idx.loc[home,'Def_EPA_per_play']), float(idx.loc[away,'Def_EPA_per_play'])
    home_pts = ((home_off - away_def) * 7 + 1.5) * 11
    away_pts = ((away_off - home_def) * 7 + 1.5) * 11
    total_base = float(home_pts + away_pts)
    # Stadium & weather
    st = str(g.get('stadium') or '')
    meta = STADIUMS.get(st) or STADIUMS.get(TEAM_HOME.get(home,''))
    roof = (meta or {}).get('roof','')
    weather_adj = 0.0
    if meta and str(roof).lower() in ['outdoor','open','none'] and pd.notna(g['start_time']):
        info = open_meteo_weather(meta['lat'], meta['lon'], pd.to_datetime(g['start_time'], utc=True))
        if info:
            if info['max_wind_mph'] > WIND_THRESHOLD: weather_adj -= TOTALS_WEATHER_ADJ
            if info['max_precip_prob'] > PRECIP_THRESHOLD: weather_adj -= TOTALS_WEATHER_ADJ
    total_final = total_base + weather_adj
    rows.append({
        'Game': f"{away} at {home}",
        'Kickoff_UTC': g['start_time'],
        'Stadium': st if st else TEAM_HOME.get(home,''),
        'Roof': roof,
        'Home_Team': home, 'Away_Team': away,
        'Spread_Base_(Home +)': spread_base,
        'Spread_Final_(Home +)': spread_final,
        'Injury_Spread_Shift': injury_shift,
        'Total_Base': total_base,
        'Weather_Adj': weather_adj,
        'Total_Final': total_final,
    })
projections = pd.DataFrame(rows)
display(projections.head())


## 🔎 6) Summary — biggest edges

In [ ]:
if projections.empty:
    display(HTML('<b>No games found for the selected week.</b>'))
else:
    top_injury = projections.assign(abs_shift=projections['Injury_Spread_Shift'].abs()).sort_values('abs_shift', ascending=False).head(3)
    top_weather = projections.assign(abs_w=projections['Weather_Adj'].abs()).sort_values('abs_w', ascending=False).head(3)
    def simple_list(df, key):
        return '<br>'.join([f"{r['Game']}: {r[key]:+.1f}" for _,r in df.iterrows()])
    html = f'''
    <div style='display:flex;gap:20px;flex-wrap:wrap'>
      <div style='flex:1;min-width:260px;padding:12px;border:1px solid #ddd;border-radius:10px'>
        <h4 style='margin:0 0 8px 0'>Top Injury Spread Shifts</h4>
        <div style='color:#b00020'>{simple_list(top_injury, 'Injury_Spread_Shift')}</div>
      </div>
      <div style='flex:1;min-width:260px;padding:12px;border:1px solid #ddd;border-radius:10px'>
        <h4 style='margin:0 0 8px 0'>Top Weather Total Drops</h4>
        <div style='color:#1a73e8'>{simple_list(top_weather, 'Weather_Adj')}</div>
      </div>
    </div>
    '''
    display(HTML(html))


## 🖼️ 7) In‑notebook tables (sortable + color‑coded)

In [ ]:
def style_projections(df):
    def row_style(row):
        styles=[]
        if abs(row['Injury_Spread_Shift'])>=1.5: styles.append('background-color:#ffd6d6')  # red-ish
        if row['Weather_Adj']!=0: styles.append('background-color:#d7e8ff')                # blue-ish
        if str(row['Roof']).lower() in ['dome','retractable']:
            styles.append('color:#666;font-style:italic')
        return ';'.join(styles) if styles else ''
    return df.style.apply(lambda r: [row_style(r)]*len(r), axis=1).format({'Spread_Base_(Home +)': '{:+.1f}','Spread_Final_(Home +)': '{:+.1f}','Injury_Spread_Shift':'{:+.1f}','Total_Base':'{:.1f}','Weather_Adj':'{:+.1f}','Total_Final':'{:.1f}'})

def style_ratings(df):
    return df.style.format({'Base_Power_Rating':'{:+.2f}','Injury_Adjustment':'{:+.2f}','Final_Power_Rating':'{:+.2f}'})

def style_injuries(df):
    return df.style.format({'Adj':'{:+.1f}'})

if 'projections' in globals():
    try:
        from google.colab import data_table
        print('▶ Sortable views (tap column headers to sort):')
        display(projections); display(ratings); display(inj_details if not inj_details.empty else pd.DataFrame(columns=['Team','Player','Pos','Status','Adj']))
    except Exception:
        pass
    print('▶ Color‑coded views:')
    display(style_projections(projections))
    display(style_ratings(ratings))
    display(style_injuries(inj_details if not inj_details.empty else pd.DataFrame(columns=['Team','Player','Pos','Status','Adj'])))


## 💾 8) Export Excel (3 tabs) + Download link

In [ ]:
today = datetime.datetime.utcnow().strftime('%Y-%m-%d')
excel_path = f'NFL_Picks_W{WEEK:02d}_{today}.xlsx'
csv_log   = f'final_power_ratings_{SEASON}_week_{WEEK}.csv'
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    ratings.to_excel(writer, sheet_name='Team_Ratings', index=False)
    projections.to_excel(writer, sheet_name='Matchup_Projections', index=False)
    (inj_details if not inj_details.empty else pd.DataFrame(columns=['Team','Player','Pos','Status','Adj']))\
        .to_excel(writer, sheet_name='Injury_Details', index=False)
ratings[['Team','Base_Power_Rating','Injury_Adjustment','Final_Power_Rating']].to_csv(csv_log, index=False)

# Highlight big injury shifts in Excel
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
wb = load_workbook(excel_path)
ws = wb['Matchup_Projections']
fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')
hdr = {c.value:i+1 for i,c in enumerate(ws[1])}
col = hdr.get('Injury_Spread_Shift')
if col:
    for r in range(2, ws.max_row+1):
        v = ws.cell(row=r, column=col).value
        try:
            if abs(float(v))>=1.5:
                for c in range(1, ws.max_column+1): ws.cell(row=r, column=c).fill = fill
        except Exception: pass
wb.save(excel_path)

print('✅ File saved:', excel_path)
print('✅ Ratings log:', csv_log)
try:
    from google.colab import files
    display(HTML(f"<p>📥 <a href='files/{excel_path}' target='_blank'>Click here to download the Excel file</a></p>"))
    files.download(excel_path)
    files.download(csv_log)
except Exception:
    pass


### Tips
- Leave week on **Auto** for current week; change dropdown to run past/future weeks.
- If NFL.com injury markup changes mid-season, injuries may be empty for a bit; try again later or tweak the scraper URLs.
